# import

In [2]:
import sys
sys.path.append('.')
import os,sys
import h5py
import numpy as np
from scipy import signal
import time
import skimage
import skimage.io, skimage.transform
from skimage.transform import resize
from skimage.util import view_as_windows
import scipy.misc
import scipy.io as sio
from utils import *
from skimage import img_as_uint
import matplotlib.pyplot as plt
# import pandas
import glob 
import datetime
import math
from skimage.color import rgb2ycbcr 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pywt
# import cv2
import re
from PIL import Image
import os
import bisect
import pickle
from PIL import Image

from sys import argv
import scipy.io as sio
from time import time
import numpy as np
import skimage.morphology as ski
from feat_spam.spam_np import getParams as getSpamParams
# from feat_spam.spam_np import getSpam
from utility.utilityImage import img2grayf

from utility.utilityImage import imread2f
from feat_spam.residue import getFiltersResidue
import feat_spam.mapping as spam_m
from scipy.signal import correlate2d
from skimage import io,data,color
import matplotlib.pyplot as plt

## convert GT's black and white area

In [4]:
NC16_IMG_DIR_splice = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_images'
NC16_IMG_MASK_splice = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_masks'


NC16_imgS = []
NC16_labelS = []

# NC16_imgS = np.ndarray([img_num_toal, 256,256,3], dtype=np.float32)
# NC16_labelS = np.ndarray([img_num_toal, 256,256], dtype=np.float32)


for i, img in enumerate(glob.glob(NC16_IMG_DIR_splice+'/*')):
#     print('\ri: %d\t%s'%(i+1, os.path.basename(img)), end='', flush=True)
    sys.stdout.write('\ri: %d\t%s'%(i+1, os.path.basename(img)))
    sys.stdout.flush()
    
    mask_mani = plt.imread('/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_masks/'+os.path.basename(img))
    
    class_labels_tensor = np.float32(mask_mani==0)
    plt.imsave(fname='/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_masks_1/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
    
   

i: 292	110.png

In [6]:

NC16_IMG_DIR_removal = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_images'
NC16_IMG_MASK_DIR_removal = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_masks'



# NC16_imgS = np.ndarray([img_num_toal, 256,256,3], dtype=np.float32)
# NC16_labelS = np.ndarray([img_num_toal, 256,256], dtype=np.float32)


for i, img in enumerate(glob.glob(NC16_IMG_DIR_removal+'/*')):
#     print('\ri: %d\t%s'%(i+1, os.path.basename(img)), end='', flush=True)
    sys.stdout.write('\ri: %d\t%s'%(i+1, os.path.basename(img)))
    sys.stdout.flush()
    
    mask_mani = plt.imread('/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_masks/'+os.path.basename(img))
    
    class_labels_tensor = np.float32(mask_mani==0)
    plt.imsave(fname='/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_masks_1/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
    

i: 208	110.png

# define the division of training and testing

In [13]:
def train_test_split1(images, labels):
    """ 将图片和标签划分为训练集和测试集
    
    :params images: 图像。ndarray。
    :params labels: 标签。ndaary。
    :params train_ratio: 训练集比例
    :params test_ratio: 测试集比例
    :return: 
    """
    train_ratio = 0.72
    RANDOM_SEED = 100
    np.random.seed(RANDOM_SEED)
    image_num = len(images)
    print('image_num =', len(images))
    rand_idx = np.arange(image_num)
    np.random.shuffle(rand_idx)
    train_idx = rand_idx[:int(train_ratio * image_num)]
#     print('train_idx=',train_idx)
    print(train_idx.dtype)
    train_idx = train_idx.astype(np.int32)
    print(train_idx.dtype)
    test_idx = rand_idx[int(train_ratio * image_num):]
    train_images = images[train_idx]
    train_labels = labels[train_idx]
    
    test_images = images[test_idx]
    test_labels = labels[test_idx]
    return train_images, train_labels,  test_images, test_labels


## wavelet resampling

In [1]:
def feature_computer(p):
    Con=0.0
    Eng=0.0
    Asm=0.0
    Idm=0.0
    Im=0.0
    
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
#             print(p[i][j])
            Con+=(i-j)*(i-j)*p[i][j]
           
            Idm+=p[i][j]/(1+(i-j)*(i-j))
            if i!=j:
                Im=p[i][j]/abs(i-j)
            if p[i][j]>0.0:
                Eng+=p[i][j]*math.log(p[i][j])
    
    return Con,-Eng,Idm,Im

In [29]:
def reample_feature_computer(imgs):
    print('np.shape(imgs)[0]=',np.shape(imgs)[0])
    for q in range(0,np.shape(imgs)[0]):

        im=imgs[q]
        img_feat = np.ndarray([np.shape(imgs)[0], 64,210], dtype=np.float32)
        # extract square patches with stride (step) 8
        patchsize=32
        # reshape to a list of patches
        rgb_patches = view_as_windows(im,(32,32,3),32)
        rgb_patches = np.squeeze(rgb_patches)
        listofpatches = np.reshape(rgb_patches,(64,32,32,3))
        
        for xx in range(0,np.shape(listofpatches)[0]):
            img = listofpatches[xx,...]
            img = rgb2ycbcr(img)[:,:,:]
            
            feature_total = np.ndarray([np.shape(listofpatches)[0],210],dtype = np.float32)
            # YYYY --- #db1----------------------
            cA_1,(cH_1,cV_1,cD_1)=pywt.dwt2(img[:,:,0], 'db1')

            feature1=np.hstack((cH_1.mean(),cH_1.std(),sum(sum(np.square(cH_1))),feature_computer(cH_1),
                               cV_1.mean(),cV_1.std(),sum(sum(np.square(cV_1))),feature_computer(cV_1),
                               cD_1.mean(),cD_1.std(),sum(sum(np.square(cD_1))),feature_computer(cD_1)))


            cA_2,(cH_2,cV_2,cD_2)=pywt.dwt2(cA_1, 'db1')
            feature2=np.hstack((cH_2.mean(),cH_2.std(),sum(sum(np.square(cH_2))),feature_computer(cH_2),
                               cV_2.mean(),cV_2.std(),sum(sum(np.square(cV_2))),feature_computer(cV_2),
                               cD_2.mean(),cD_2.std(),sum(sum(np.square(cD_2))),feature_computer(cD_2)))

            cA_3,(cH_3,cV_3,cD_3)=pywt.dwt2(cA_2, 'db1')
            feature3=np.hstack((cA_3.mean(),cA_3.std(),sum(sum(np.square(cA_3))),feature_computer(cA_3),
                               cH_3.mean(),cH_3.std(),sum(sum(np.square(cH_3))),feature_computer(cH_3),
                               cV_3.mean(),cV_3.std(),sum(sum(np.square(cV_3))),feature_computer(cV_3),
                               cD_3.mean(),cD_3.std(),sum(sum(np.square(cD_3))),feature_computer(cD_3)))
            
            # CbCbCb --- #db1----------------------
            cA_1,(cH_1,cV_1,cD_1)=pywt.dwt2(img[:,:,1], 'db1')
            feature4=np.hstack((cH_1.mean(),cH_1.std(),sum(sum(np.square(cH_1))),feature_computer(cH_1),
                               cV_1.mean(),cV_1.std(),sum(sum(np.square(cV_1))),feature_computer(cV_1),
                               cD_1.mean(),cD_1.std(),sum(sum(np.square(cD_1))),feature_computer(cD_1)))

            cA_2,(cH_2,cV_2,cD_2)=pywt.dwt2(cA_1, 'db1')
            feature5=np.hstack((cH_2.mean(),cH_2.std(),sum(sum(np.square(cH_2))),feature_computer(cH_2),
                               cV_2.mean(),cV_2.std(),sum(sum(np.square(cV_2))),feature_computer(cV_2),
                               cD_2.mean(),cD_2.std(),sum(sum(np.square(cD_2))),feature_computer(cD_2)))

            cA_3,(cH_3,cV_3,cD_3)=pywt.dwt2(cA_2, 'db1')
            feature6=np.hstack((cA_3.mean(),cA_3.std(),sum(sum(np.square(cA_3))),feature_computer(cA_3),
                               cH_3.mean(),cH_3.std(),sum(sum(np.square(cH_3))),feature_computer(cH_3),
                               cV_3.mean(),cV_3.std(),sum(sum(np.square(cV_3))),feature_computer(cV_3),
                               cD_3.mean(),cD_3.std(),sum(sum(np.square(cD_3))),feature_computer(cD_3)))
                    # CrCrCr--- #db1----------------------
            cA_1,(cH_1,cV_1,cD_1)=pywt.dwt2(img[:,:,2], 'db1')
            feature7=np.hstack((cH_1.mean(),cH_1.std(),sum(sum(np.square(cH_1))),feature_computer(cH_1),
                               cV_1.mean(),cV_1.std(),sum(sum(np.square(cV_1))),feature_computer(cV_1),
                               cD_1.mean(),cD_1.std(),sum(sum(np.square(cD_1))),feature_computer(cD_1)))

            cA_2,(cH_2,cV_2,cD_2)=pywt.dwt2(cA_1, 'db1')
            feature8=np.hstack((cH_2.mean(),cH_2.std(),sum(sum(np.square(cH_2))),feature_computer(cH_2),
                               cV_2.mean(),cV_2.std(),sum(sum(np.square(cV_2))),feature_computer(cV_2),
                               cD_2.mean(),cD_2.std(),sum(sum(np.square(cD_2))),feature_computer(cD_2)))

            cA_3,(cH_3,cV_3,cD_3)=pywt.dwt2(cA_2, 'db1')
            feature9=np.hstack((cA_3.mean(),cA_3.std(),sum(sum(np.square(cA_3))),feature_computer(cA_3),
                               cH_3.mean(),cH_3.std(),sum(sum(np.square(cH_3))),feature_computer(cH_3),
                               cV_3.mean(),cV_3.std(),sum(sum(np.square(cV_3))),feature_computer(cV_3),
                               cD_3.mean(),cD_3.std(),sum(sum(np.square(cD_3))),feature_computer(cD_3)))
            
            
            feature=np.hstack((feature1,feature2,feature3,feature4,feature5,feature6,feature7,\
                               feature8,feature9))
#             print(feature.shape)
            feature_total[xx,...] =  feature
        #
        img_feat[q, ...] = feature_total
#     img_feat = np.array(img_feat, dtype=np.float32)
#     print('img_feat_shape=',img_feat.shape)
        print ("feature extrating for image # "+ str(q+1)+", with shape-->"+str(np.shape(feature_total)))
    return img_feat

# extract resampling features

In [8]:
def reample_feature_computer(imgs):
    print('np.shape(imgs)[0]=',np.shape(imgs)[0])
    for q in range(0,np.shape(imgs)[0]):
        sys.stdout.write('\r' + str(q))
        sys.stdout.flush()
        im=imgs[q]
        img_feat = np.ndarray([np.shape(imgs)[0], 64,240], dtype=np.float32)
        # extract square patches with stride (step) 8
        patchsize=32
        # reshape to a list of patches
        rgb_patches = view_as_windows(im,(32,32,3),32)
        rgb_patches = np.squeeze(rgb_patches)
        listofpatches = np.reshape(rgb_patches,(64,32,32,3))
        #listofpatches = orig_object.reshape((1, patchsize, patchsize, 3))
        #print("patches array reshaped to list of patches with shape "+str(listofpatches.shape))

        # Radon projection parameters
        circle_inscribed = False
        numAngles = 10
        theta = np.linspace(0,180,numAngles,endpoint=False)

        def radon_projections_compiled_cuda(patches, thetas, circle_inscribed):
            sys.path.append(os.path.join(thispath,'build')) # for importing pysinogram.so
            from pysinogram import BatchRadonTransform
            return np.array(BatchRadonTransform(list(patches), list(thetas), circle_inscribed))

        def radon_projections_skimage_python(patches, thetas, circle_inscribed):
            # sqrt(abs(  2D discrete 3x3 laplacian filter  ))
            kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
            laplacefilter = lambda xx: np.sqrt(np.fabs(signal.convolve2d(xx, kernel, mode='same', boundary='symm')))
            # do laplacian filter on each channel independently, then average across channels
            rgbfilter = lambda xx: np.mean([laplacefilter(xx[:,:,chan]) for chan in range(xx.shape[2])], axis=0)
            # transpose the sinogram output, to be consistent with CUDA implementation above
            myradon = lambda xx: skimage.transform.radon(rgbfilter(xx), theta=theta, circle=circle_inscribed).transpose()
            # iterate processing over all patches
            return np.stack([myradon(patches[ii,...]) for ii in range(patches.shape[0])], axis=0)

        # run tests
        if False:
            # compare against compiled implementation
            # requires compiling using cuda-radon-transform repository, available on Bitbucket
            t0 = time.time()
            check11 =  radon_projections_compiled_cuda(listofpatches, theta, circle_inscribed)
            t1 = time.time()
            check22 = radon_projections_skimage_python(listofpatches, theta, circle_inscribed)
            t2 = time.time()
            print("Radon projections time, compiled CUDA:  "+str(t1-t0)+" seconds")
            print("Radon projections time, python skimage: "+str(t2-t1)+" seconds")
            describe("check11", check11)
            describe("check22", check22)

            import cv2
            for ii in range(check11.shape[0]):
                checkdiff = np.fabs(check11[ii,:,:] - check22[ii,:,:])
                describe("checkdiff", checkdiff)
                zp = np.zeros((4,check11.shape[2]))
                concat = np.concatenate((check11[ii,:,:], zp, check22[ii,:,:], zp, checkdiff), axis=0)
                #cv2.imshow("npresult", uint8norm(concat))
                #cv2.waitKey(0)
        else:
            # run only one of the implementations
            radonfunc = radon_projections_skimage_python
#             beftime = time.time()
            npresult = radonfunc(listofpatches, theta, circle_inscribed)
            #print("sinogram calculation took "+str(time.time()-beftime)+" seconds")
            #describe("python sinogram", npresult)
            assert len(npresult.shape) == 3, str(npresult.shape)

            # also do FFT + normalization as final stage of feature extraction
            # subtract 1 from normed which is the mean

            absproc = lambda xx: np.expand_dims(np.absolute(xx), axis=-1)
#             beftime = time.time()
            _, fftnormed, _, fftavg = fftscores(npresult)

            npresult = absproc(fftnormed) - 1.
            #npresult = np.concatenate([absproc(fftnormed) - 1., absproc(fftavg)], axis=1)
            npresult=np.transpose(npresult,(3,0,1,2))

            npresult=np.reshape(npresult,(64,240))
            print ("feature extrating for image # "+ str(q+1)+", with shape-->"+str(np.shape(npresult)))
            #print("FFT calculations took "+str(time.time()-beftime)+" seconds")
            #describe("npresult", npresult)
#             hdf5_file["feat"][q, ...] = npresult[None]
#             img_feat.append(npresult)
            img_feat[q, ...] = npresult[None]
#     img_feat = np.array(img_feat, dtype=np.float32)
    print('img_feat_shape=',img_feat.shape)
    return img_feat

# extract the features of the co-occurrence matrix

In [8]:
defaultSpamParam = getSpamParams(ordResid=3, symTranspose=True, q=2, T=1, ordCooc=4, mapper='SignSym', strides=8)
smallSpamParam   = getSpamParams(ordResid=3, symTranspose=True, q=2, T=1, ordCooc=4, mapper='SignSym', strides=4)
def SB_main(imgfilename):
    img = imgfilename
    imgsize = img.shape
    print('imgsize:',imgsize)
    if imgsize[0]*imgsize[1]>20000:
         mapp = SBgu(img, paramSpam = defaultSpamParam, ksize = 128, flagDropSaturated=2, paddingModality=0)
    else:
         mapp = SBgu(img, paramSpam = smallSpamParam, ksize =  64, flagDropSaturated=2, paddingModality=0)
    
    return mapp

def SBgu(img, paramSpam = defaultSpamParam, ksize = 128, flagDropSaturated = 2, satutationProb = 0.85, extFeat = range(25), seed = 0, maxIter = 100, replicates = 30, outliersNlogl = 42, paddingModality = 0, flagShow = False):
    if flagDropSaturated > 0:
        if img.dtype== np.uint8:
            weights = np.logical_not(getSatMap(img, 6, 252)).astype(np.float32)
        else:
            weights = np.logical_not(getSatMap(img, 6.0 / 256, 252.0 / 256)).astype(np.float32)
        weights = ski.binary_erosion(weights, np.ones((2*paramSpam['radius'][0]+1, 2*paramSpam['radius'][1]+1), dtype = np.bool))
    else:
        weights = list()
    
#     img = img2grayf(img)
    img = color.rgb2hsv(img)
    img1 = img[:,:,0]
    img2 = img[:,:,1]
    img3 = img[:,:,2]
#     print('img_size:',img.shape)
    spam1 = getSpam(img1, paramSpam, ksize, weights = weights, paddingModality = paddingModality)
    spam2 = getSpam(img2, paramSpam, ksize, weights = weights, paddingModality = paddingModality)
    spam3 = getSpam(img3, paramSpam, ksize, weights = weights, paddingModality = paddingModality)
    return spam1,spam2,spam3

def getSpam(X, params, ksize, weights = list(), paddingModality = 0):

    strides = params['strides']
    if isinstance(ksize, (list,tuple)):
        ksize = ksize[0:2]
    else:
        ksize = [ksize, ksize]

    ksize[0] = int(ksize[0] / strides[0])
    ksize[1] = int(ksize[1] / strides[1])

    spam = computeSpam(X, params, weights = weights, normalize=True)
    
    return spam


def getSatMap(img, th_Black, th_White):
    if len(img.shape) > 2:
        sat_mask0 = ski.binary_opening(img[:,:,0] < th_Black, ski.disk(3))
        sat_mask1 = ski.binary_opening(img[:,:,0] > th_White, ski.disk(2))
        for index in range(1, img.shape[2]):
            sat_mask0 = np.logical_or(sat_mask0, ski.binary_opening(img[:, :, index] < th_Black, ski.disk(3)))
            sat_mask1 = np.logical_or(sat_mask1, ski.binary_opening(img[:, :, index] > th_White, ski.disk(2)))
    else:
        sat_mask0 = ski.binary_opening(img<th_Black,ski.disk(3))
        sat_mask1 = ski.binary_opening(img>th_White,ski.disk(2))

    return np.logical_or(sat_mask0,sat_mask1)

def computeSpam(X, params, weights = list(),  normalize = True):
    
#     print('X:',X.shape)
    ## Residue
    WresH = params['Wres']
    WresV = WresH.transpose([1,0,2,3])
    resH = correlate2d(X.squeeze(), WresH.squeeze(), mode='valid')
    resV = correlate2d(X.squeeze(), WresV.squeeze(), mode='valid')
#     print('resH:',resH.shape)
    ## Quantization & Truncation
    values = params['values']
    if params['uniformQuant']:
        n = values.size
        T = (n - 1) / 2
        q = (values[1] - values[0])
        resHq = (np.clip(my_round(resH / q) + T, 0, n - 1)).astype(np.int64)
        resVq = (np.clip(my_round(resV / q) + T, 0, n - 1)).astype(np.int64)
    else:
        resHq = quantizerScalarEncoder(resH, values)
        resVq = quantizerScalarEncoder(resV, values)
    
#     print('resHq:',resHq.shape)
    ## Coocorance
    ordCooc = params['ordCooc']
    n = (params['values']).size
    dim = int(ordCooc + 1 - (ordCooc % 2))
    indexL = int((dim - 1) / 2)
#     indexL =0
    shapeR = np.asarray(resHq.shape[:2]) - dim +1 
#     shapeR = np.asarray(resHq.shape)
#     print('shapeR:',shapeR)
    resHh = np.zeros(shapeR, dtype = np.int)
    resVh = np.zeros(shapeR, dtype = np.int)
    resHv = np.zeros(shapeR, dtype = np.int)
    resVv = np.zeros(shapeR, dtype = np.int)

    for indexP in range(ordCooc):
        nn = (n ** indexP)
        
#         print('n---:', n)
#         print('indexp==:', indexP)
#         print('===nn:',nn)
#         print(indexL,(shapeR[0] + indexL))
#         print(indexP,(shapeR[1] + indexP))
        
        resHh += resHq[indexL:(shapeR[0] + indexL), indexP:(shapeR[1] + indexP)] * nn
        resVh += resVq[indexL:(shapeR[0] + indexL), indexP:(shapeR[1] + indexP)] * nn
        resHv += resHq[indexP:(shapeR[0] + indexP), indexL:(shapeR[1] + indexL)] * nn
        resVv += resVq[indexP:(shapeR[0] + indexP), indexL:(shapeR[1] + indexL)] * nn
#     print('resHh:',resHh.shape)
#     print('resVh:',resVh.shape)
#     print('resHv:',resHv.shape)
#     print('resVv:',resVv.shape)
#     print('resHh:',resHh+resVh+resHv+resVv)
    ## Mappeing
    mapper = params['mapper']
    if len(mapper) > 0:
        resHh = mapper['table'][resHh].squeeze()
        resVh = mapper['table'][resVh].squeeze()
        resHv = mapper['table'][resHv].squeeze()
        resVv = mapper['table'][resVv].squeeze()
#     print('resHh:',resHh.shape)
#     print('resVh:',resVh.shape)
#     print('resHv:',resHv.shape)
#     print('resVv:',resVv.shape)
    
    
    co_occurrence = resHh+resVh+resHv+resVv
#     print('co_occurrence:',co_occurrence.shape)
    return co_occurrence

In [9]:
my_round = lambda x: np.sign(x) * np.floor(np.abs(x)+0.5)
def reample_feature_computer1(imgs):
    print('np.shape(imgs)[0]=',np.shape(imgs)[0])
    for q in range(0,np.shape(imgs)[0]):
        sys.stdout.write('\r' + str(q))
        sys.stdout.flush()
        im=imgs[q]
        img_feat = np.ndarray([np.shape(imgs)[0], 248,248,3], dtype=np.float32)
        my_round = lambda x: np.sign(x) * np.floor(np.abs(x)+0.5)
        
        mapp1,mapp2,mapp3 = SB_main(im)

        map = np.stack((mapp1,mapp2,mapp3),axis =2)
        img_feat[q, ...] = map[None]
    print('img_feat_shape=',img_feat.shape)
    return img_feat

# load dataset

## NC2016

In [10]:
NC16_IMG_DIR_manipulation = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/manipulation_images'
NC16_IMG_MASK_DIR_manipulation = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/manipulation_masks'
mani_img_num = len(glob.glob(NC16_IMG_DIR_manipulation+'/*'))
mani_gt_num = len(glob.glob(NC16_IMG_MASK_DIR_manipulation+'/*'))

NC16_IMG_DIR_removal = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_images'
NC16_IMG_MASK_DIR_removal = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_masks'
removal_img_num = len(glob.glob(NC16_IMG_DIR_removal+'/*'))
removal_gt_num = len(glob.glob(NC16_IMG_MASK_DIR_removal+'/*'))

NC16_IMG_DIR_splice = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_images'
NC16_IMG_MASK_splice = '/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_masks'
splice_img_num = len(glob.glob(NC16_IMG_DIR_splice+'/*'))
splice_gt_num = len(glob.glob(NC16_IMG_MASK_splice+'/*'))

img_num_toal = (mani_img_num+removal_img_num+splice_img_num)
gt_num_toal = (mani_gt_num+removal_gt_num+splice_gt_num)

print('img_num_toal = ',img_num_toal)#一共1064幅
print('gt_num_toal = ',gt_num_toal)

NC16_imgS = []
NC16_labelS = []

# NC16_imgS = np.ndarray([img_num_toal, 256,256,3], dtype=np.float32)
# NC16_labelS = np.ndarray([img_num_toal, 256,256], dtype=np.float32)


for i, img in enumerate(glob.glob(NC16_IMG_DIR_manipulation+'/*')):
#     print('\ri: %d\t%s'%(i+1, os.path.basename(img)), end='', flush=True)
    sys.stdout.write('\ri: %d\t%s'%(i+1, os.path.basename(img)))
    sys.stdout.flush()
    
#     image_mani = plt.imread(img)
#     print(type(image_mani))
#     image_mani = image_mani.resize((256,256,3), Image.ANTIALIAS)
    image_mani=scipy.misc.imresize(plt.imread(img),(256,256,3),interp='bilinear',mode=None)
    plt.imsave(fname='../data_256/NC_16/manipulation/img/'+os.path.basename(img),arr=image_mani)
    
    mask_mani = plt.imread('/home/shizenan/Datasets/NC2016_Test/data_by_shi/manipulation_masks/'+os.path.basename(img))
    mask_mani = scipy.misc.imresize(mask_mani,(256,256),interp='bilinear',mode=None)
    class_labels_tensor = np.float32(mask_mani==0)
    plt.imsave(fname='../data_256/NC_16/manipulation/GT/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
    
    NC16_imgS.append(image_mani)
    NC16_labelS.append(class_labels_tensor)

for i, img in enumerate(glob.glob(NC16_IMG_DIR_removal+'/*')):
#     print('\ri: %d\t%s'%(mani_img_num+i+1, os.path.basename(img)), end='', flush=True)
    sys.stdout.write('\ri: %d\t%s'%(mani_img_num+i+1, os.path.basename(img)))
    sys.stdout.flush()
    
#     image_removal = plt.imread(img)
#     image_removal = image_removal.resize((256,256,3), Image.ANTIALIAS)
    image_removal=scipy.misc.imresize(plt.imread(img),(256,256,3),interp='bilinear',mode=None)
    plt.imsave(fname='../data_256/NC_16/removal/img/'+os.path.basename(img),arr=image_removal)
    mask_removal = plt.imread('/home/shizenan/Datasets/NC2016_Test/data_by_shi/removal_masks/'+os.path.basename(img))
#     mask_removal = mask_removal.imresize((256,256),Image.ANTIALIAS)
    mask_removal = scipy.misc.imresize(mask_removal,(256,256),interp='bilinear',mode=None)
    
    class_labels_tensor = np.float32(mask_removal==0)
    plt.imsave(fname='../data_256/NC_16/removal/GT/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
   
    NC16_imgS.append(image_removal)
    NC16_labelS.append(class_labels_tensor)


for i, img in enumerate(glob.glob(NC16_IMG_DIR_splice+'/*')):
#     print('\ri: %d\t%s'%(mani_img_num+removal_img_num+i+1, os.path.basename(img)), end='', flush=True)
    sys.stdout.write('\ri: %d\t%s'%(mani_img_num+removal_img_num+i+1, os.path.basename(img)))
    sys.stdout.flush()
#     image_splice = plt.imread(img)
#     image_splice = image_splice.resize((256,256,3), Image.ANTIALIAS)
#     mask_splice = plt.imread('/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_masks/'+os.path.basename(img))
#     mask_splice = mask_splice.imresize((256,256),interp='bilinear',mode=None) 
    
    image_splice = scipy.misc.imresize(plt.imread(img),(256,256,3),interp='bilinear',mode=None)
    plt.imsave(fname='../data_256/NC_16/splice/img/'+os.path.basename(img),arr=image_splice)
    mask_splice = plt.imread('/home/shizenan/Datasets/NC2016_Test/data_by_shi/splice_masks/'+os.path.basename(img))
    mask_splice = scipy.misc.imresize(mask_splice,(256,256),interp='bilinear',mode=None)
    class_labels_tensor = np.float32(mask_splice==0)
    plt.imsave(fname='../data_256/NC_16/splice/GT/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
    
    NC16_imgS.append(image_splice)
    NC16_labelS.append(class_labels_tensor)

# NC16_train_img, NC16_train_label, NC16_vali_img, NC16_vali_label, NC16_test_img, \
# NC16_test_label=  train_vali_test_split(NC16_imgS, NC16_labelS, 0.7, 0.05)   
NC16_imgS = np.array(NC16_imgS,dtype = np.float32)
NC16_labelS = np.array(NC16_labelS,dtype = np.float32)
np.save('NC16_mani_imgS.npy', NC16_imgS)
np.save('NC16_mani_labelS.npy', NC16_labelS)
print(len(NC16_imgS))
np.save('NC16_imgS.npy', NC16_imgS)
np.save('NC16_labelS.npy', NC16_labelS)

NC16_train_img,  NC16_train_label, NC16_test_img, NC16_test_label= train_test_split1\
(NC16_imgS, NC16_labelS)

print()
NC16_train_img_num = len(NC16_train_img)
NC16_train_label_num = len(NC16_train_label)
print('NC16_train_img.shape',NC16_train_img.shape)
print('NC16_train_label.shape',NC16_train_label.shape)

NC16_test_img_num = len(NC16_test_img)
NC16_test_label_num = len(NC16_test_label)
print('NC16_test_img_num.shape',NC16_test_img.shape)
print('NC16_test_label.shape',NC16_test_label.shape)

print(len(NC16_train_label)+len(NC16_test_label))    
print()

np.save('../dataset_npy/NC_16/train/NC16_train_img1.npy', NC16_train_img)
np.save('../dataset_npy/NC_16/train/NC16_train_label1.npy', NC16_train_label)
np.save('../dataset_npy/NC_16/test/NC16_test_img1.npy', NC16_test_img)
np.save('../dataset_npy/NC_16/test/NC16_test_label1.npy', NC16_test_label)

img_num_toal =  1064
gt_num_toal =  1064
i: 1	279.png

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


i: 2	103.png

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


i: 564	499.png564


## reample_feature_compute

In [17]:
NC16_img = np.load('../dataset_npy/NC_16/mani/NC16_mani_train_img.npy')
NC16_img_feat = reample_feature_computer1(NC16_img)

np.save('../dataset_npy/NC_16/mani/NC16_mani__train_imgS_feat.npy', NC16_img_feat)


np.shape(imgs)[0]= 406
0imgsize: (256, 256, 3)
1imgsize: (256, 256, 3)
2imgsize: (256, 256, 3)
3imgsize: (256, 256, 3)
4imgsize: (256, 256, 3)
5imgsize: (256, 256, 3)
6imgsize: (256, 256, 3)
7imgsize: (256, 256, 3)
8imgsize: (256, 256, 3)
9imgsize: (256, 256, 3)
10imgsize: (256, 256, 3)
11imgsize: (256, 256, 3)
12imgsize: (256, 256, 3)
13imgsize: (256, 256, 3)
14imgsize: (256, 256, 3)
15imgsize: (256, 256, 3)
16imgsize: (256, 256, 3)
17imgsize: (256, 256, 3)
18imgsize: (256, 256, 3)
19imgsize: (256, 256, 3)
20imgsize: (256, 256, 3)
21imgsize: (256, 256, 3)
22imgsize: (256, 256, 3)
23imgsize: (256, 256, 3)
24imgsize: (256, 256, 3)
25imgsize: (256, 256, 3)
26imgsize: (256, 256, 3)
27imgsize: (256, 256, 3)
28imgsize: (256, 256, 3)
29imgsize: (256, 256, 3)
30imgsize: (256, 256, 3)
31imgsize: (256, 256, 3)
32imgsize: (256, 256, 3)
33imgsize: (256, 256, 3)
34imgsize: (256, 256, 3)
35imgsize: (256, 256, 3)
36imgsize: (256, 256, 3)
37imgsize: (256, 256, 3)
38imgsize: (256, 256, 3)
39imgsize: (

319imgsize: (256, 256, 3)
320imgsize: (256, 256, 3)
321imgsize: (256, 256, 3)
322imgsize: (256, 256, 3)
323imgsize: (256, 256, 3)
324imgsize: (256, 256, 3)
325imgsize: (256, 256, 3)
326imgsize: (256, 256, 3)
327imgsize: (256, 256, 3)
328imgsize: (256, 256, 3)
329imgsize: (256, 256, 3)
330imgsize: (256, 256, 3)
331imgsize: (256, 256, 3)
332imgsize: (256, 256, 3)
333imgsize: (256, 256, 3)
334imgsize: (256, 256, 3)
335imgsize: (256, 256, 3)
336imgsize: (256, 256, 3)
337imgsize: (256, 256, 3)
338imgsize: (256, 256, 3)
339imgsize: (256, 256, 3)
340imgsize: (256, 256, 3)
341imgsize: (256, 256, 3)
342imgsize: (256, 256, 3)
343imgsize: (256, 256, 3)
344imgsize: (256, 256, 3)
345imgsize: (256, 256, 3)
346imgsize: (256, 256, 3)
347imgsize: (256, 256, 3)
348imgsize: (256, 256, 3)
349imgsize: (256, 256, 3)
350imgsize: (256, 256, 3)
351imgsize: (256, 256, 3)
352imgsize: (256, 256, 3)
353imgsize: (256, 256, 3)
354imgsize: (256, 256, 3)
355imgsize: (256, 256, 3)
356imgsize: (256, 256, 3)
357imgsize: 

In [18]:
NC16_img = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_img.npy')
NC16_img_feat = reample_feature_computer1(NC16_img)

np.save('../dataset_npy/NC_16/mani/NC16_mani_test_imgS_feat.npy', NC16_img_feat)


np.shape(imgs)[0]= 158
0imgsize: (256, 256, 3)
1imgsize: (256, 256, 3)
2imgsize: (256, 256, 3)
3imgsize: (256, 256, 3)
4imgsize: (256, 256, 3)
5imgsize: (256, 256, 3)
6imgsize: (256, 256, 3)
7imgsize: (256, 256, 3)
8imgsize: (256, 256, 3)
9imgsize: (256, 256, 3)
10imgsize: (256, 256, 3)
11imgsize: (256, 256, 3)
12imgsize: (256, 256, 3)
13imgsize: (256, 256, 3)
14imgsize: (256, 256, 3)
15imgsize: (256, 256, 3)
16imgsize: (256, 256, 3)
17imgsize: (256, 256, 3)
18imgsize: (256, 256, 3)
19imgsize: (256, 256, 3)
20imgsize: (256, 256, 3)
21imgsize: (256, 256, 3)
22imgsize: (256, 256, 3)
23imgsize: (256, 256, 3)
24imgsize: (256, 256, 3)
25imgsize: (256, 256, 3)
26imgsize: (256, 256, 3)
27imgsize: (256, 256, 3)
28imgsize: (256, 256, 3)
29imgsize: (256, 256, 3)
30imgsize: (256, 256, 3)
31imgsize: (256, 256, 3)
32imgsize: (256, 256, 3)
33imgsize: (256, 256, 3)
34imgsize: (256, 256, 3)
35imgsize: (256, 256, 3)
36imgsize: (256, 256, 3)
37imgsize: (256, 256, 3)
38imgsize: (256, 256, 3)
39imgsize: (

## finetuning

In [ ]:
NC16_img = np.load('NC16_mani_train_img.npy')
NC16_img_feat = reample_feature_computer1(NC16_img)


np.save('../dataset_npy/NC_16/NC16_mani_train_img_feat.npy', NC16_img_feat)



In [14]:
NC16_mani_img = np.load('NC16_mani_imgS.npy')
NC16_mani_label = np.load('NC16_mani_labelS.npy')
# NC16_img_feat = reample_feature_computer1(NC16_img)


# np.save('../dataset_npy/NC_16/NC16_mani_imgS_feat1.npy', NC16_img_feat)


NC16_mani_train_img,  NC16_mani_train_label, NC16_mani_test_img, NC16_mani_test_label= train_test_split1\
(NC16_mani_img, NC16_mani_label)

print()
NC16_mani_train_img_num = len(NC16_mani_train_img)
NC16_mani_train_label_num = len(NC16_mani_train_label)
print('NC16_mani_train_img.shape',NC16_mani_train_img.shape)
print('NC16_mani_train_label.shape',NC16_mani_train_label.shape)

NC16_mani_test_img_num = len(NC16_mani_test_img)
NC16_mani_test_label_num = len(NC16_mani_test_label)
print('NC16_mani_test_img_num.shape',NC16_mani_test_img.shape)
print('NC16_mani_test_label.shape',NC16_mani_test_label.shape)

print(len(NC16_mani_train_label)+len(NC16_mani_test_label))  

np.save('../dataset_npy/NC_16/mani/NC16_mani_train_img.npy', NC16_mani_train_img)
np.save('../dataset_npy/NC_16/mani/NC16_mani_train_label.npy', NC16_mani_train_label)
np.save('../dataset_npy/NC_16/mani/NC16_mani_test_img.npy', NC16_mani_test_img)
np.save('../dataset_npy/NC_16/mani/NC16_mani_test_label.npy', NC16_mani_test_label)

image_num = 564
int64
int32

NC16_mani_train_img.shape (406, 256, 256, 3)
NC16_mani_train_label.shape (406, 256, 256)
NC16_mani_test_img_num.shape (158, 256, 256, 3)
NC16_mani_test_label.shape (158, 256, 256)
564


### training dataset

In [17]:
NC16_train_img = np.load('../dataset_npy/NC_16/train/NC16_train_img.npy')
NC16_train_img_feat = reample_feature_computer(NC16_train_img)
np.save('../dataset_npy/NC_16/train/NC16_train_img_cohere_feat.npy', NC16_train_img_feat)
# all_imgF = np.load('../dataset_npy/NC_16/test/NC16_test_img.npy')

('np.shape(imgs)[0]=', 798)
0('imgsize:', (256, 256, 3))
1('imgsize:', (256, 256, 3))
2('imgsize:', (256, 256, 3))
3('imgsize:', (256, 256, 3))
4('imgsize:', (256, 256, 3))
5('imgsize:', (256, 256, 3))
6('imgsize:', (256, 256, 3))
7('imgsize:', (256, 256, 3))
8('imgsize:', (256, 256, 3))
9('imgsize:', (256, 256, 3))
10('imgsize:', (256, 256, 3))
11('imgsize:', (256, 256, 3))
12('imgsize:', (256, 256, 3))
13('imgsize:', (256, 256, 3))
14('imgsize:', (256, 256, 3))
15('imgsize:', (256, 256, 3))
16('imgsize:', (256, 256, 3))
17('imgsize:', (256, 256, 3))
18('imgsize:', (256, 256, 3))
19('imgsize:', (256, 256, 3))
20('imgsize:', (256, 256, 3))
21('imgsize:', (256, 256, 3))
22('imgsize:', (256, 256, 3))
23('imgsize:', (256, 256, 3))
24('imgsize:', (256, 256, 3))
25('imgsize:', (256, 256, 3))
26('imgsize:', (256, 256, 3))
27('imgsize:', (256, 256, 3))
28('imgsize:', (256, 256, 3))
29('imgsize:', (256, 256, 3))
30('imgsize:', (256, 256, 3))
31('imgsize:', (256, 256, 3))
32('imgsize:', (256, 2

267('imgsize:', (256, 256, 3))
268('imgsize:', (256, 256, 3))
269('imgsize:', (256, 256, 3))
270('imgsize:', (256, 256, 3))
271('imgsize:', (256, 256, 3))
272('imgsize:', (256, 256, 3))
273('imgsize:', (256, 256, 3))
274('imgsize:', (256, 256, 3))
275('imgsize:', (256, 256, 3))
276('imgsize:', (256, 256, 3))
277('imgsize:', (256, 256, 3))
278('imgsize:', (256, 256, 3))
279('imgsize:', (256, 256, 3))
280('imgsize:', (256, 256, 3))
281('imgsize:', (256, 256, 3))
282('imgsize:', (256, 256, 3))
283('imgsize:', (256, 256, 3))
284('imgsize:', (256, 256, 3))
285('imgsize:', (256, 256, 3))
286('imgsize:', (256, 256, 3))
287('imgsize:', (256, 256, 3))
288('imgsize:', (256, 256, 3))
289('imgsize:', (256, 256, 3))
290('imgsize:', (256, 256, 3))
291('imgsize:', (256, 256, 3))
292('imgsize:', (256, 256, 3))
293('imgsize:', (256, 256, 3))
294('imgsize:', (256, 256, 3))
295('imgsize:', (256, 256, 3))
296('imgsize:', (256, 256, 3))
297('imgsize:', (256, 256, 3))
298('imgsize:', (256, 256, 3))
299('img

532('imgsize:', (256, 256, 3))
533('imgsize:', (256, 256, 3))
534('imgsize:', (256, 256, 3))
535('imgsize:', (256, 256, 3))
536('imgsize:', (256, 256, 3))
537('imgsize:', (256, 256, 3))
538('imgsize:', (256, 256, 3))
539('imgsize:', (256, 256, 3))
540('imgsize:', (256, 256, 3))
541('imgsize:', (256, 256, 3))
542('imgsize:', (256, 256, 3))
543('imgsize:', (256, 256, 3))
544('imgsize:', (256, 256, 3))
545('imgsize:', (256, 256, 3))
546('imgsize:', (256, 256, 3))
547('imgsize:', (256, 256, 3))
548('imgsize:', (256, 256, 3))
549('imgsize:', (256, 256, 3))
550('imgsize:', (256, 256, 3))
551('imgsize:', (256, 256, 3))
552('imgsize:', (256, 256, 3))
553('imgsize:', (256, 256, 3))
554('imgsize:', (256, 256, 3))
555('imgsize:', (256, 256, 3))
556('imgsize:', (256, 256, 3))
557('imgsize:', (256, 256, 3))
558('imgsize:', (256, 256, 3))
559('imgsize:', (256, 256, 3))
560('imgsize:', (256, 256, 3))
561('imgsize:', (256, 256, 3))
562('imgsize:', (256, 256, 3))
563('imgsize:', (256, 256, 3))
564('img

797('imgsize:', (256, 256, 3))
('img_feat_shape=', (798, 248, 248, 3))


### testing dataset

In [18]:
NC16_test_img = np.load('../dataset_npy/NC_16/test/NC16_test_img.npy')
print(NC16_test_img.shape)

NC16_test_img_feat = reample_feature_computer(NC16_test_img)
np.save('../dataset_npy/NC_16/test/NC16_test_img_cohere_feat.npy', NC16_test_img_feat)

(266, 256, 256, 3)
('np.shape(imgs)[0]=', 266)
0('imgsize:', (256, 256, 3))
1('imgsize:', (256, 256, 3))
2('imgsize:', (256, 256, 3))
3('imgsize:', (256, 256, 3))
4('imgsize:', (256, 256, 3))
5('imgsize:', (256, 256, 3))
6('imgsize:', (256, 256, 3))
7('imgsize:', (256, 256, 3))
8('imgsize:', (256, 256, 3))
9('imgsize:', (256, 256, 3))
10('imgsize:', (256, 256, 3))
11('imgsize:', (256, 256, 3))
12('imgsize:', (256, 256, 3))
13('imgsize:', (256, 256, 3))
14('imgsize:', (256, 256, 3))
15('imgsize:', (256, 256, 3))
16('imgsize:', (256, 256, 3))
17('imgsize:', (256, 256, 3))
18('imgsize:', (256, 256, 3))
19('imgsize:', (256, 256, 3))
20('imgsize:', (256, 256, 3))
21('imgsize:', (256, 256, 3))
22('imgsize:', (256, 256, 3))
23('imgsize:', (256, 256, 3))
24('imgsize:', (256, 256, 3))
25('imgsize:', (256, 256, 3))
26('imgsize:', (256, 256, 3))
27('imgsize:', (256, 256, 3))
28('imgsize:', (256, 256, 3))
29('imgsize:', (256, 256, 3))
30('imgsize:', (256, 256, 3))
31('imgsize:', (256, 256, 3))
32(